## Загрузка и подготовка данных

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical

# Загрузка данных
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Преобразование меток классов в one-hot векторы
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Нормализация данных
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Изменение формы данных для совместимости с Keras
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)




## Построение, обучение и оценка модели

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))


In [ ]:
optimizer = 'adam'
loss = 'categorical_crossentropy'
metrics = ['accuracy']
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)


In [ ]:
epochs = 10
batch_size = 128
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs)


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Точность на тестовых данных:", test_accuracy)


In [ ]:
pred = model.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X_test[0])

In [ ]:
y_test[0]

In [ ]:
pred[0]

In [ ]:
import numpy as np

# Создаем список с русскими названиями классов
class_names_rus = [
    "Футболка/топ",
    "Брюки",
    "Пуловер",
    "Платье",
    "Пальто",
    "Сандали",
    "Рубашка",
    "Кроссовки",
    "Сумка",
    "Ботинки"
]

# Возьмем первое предсказание из тестовых данных

# Находим индекс максимального значения в массиве с помощью argmax()
class_index = np.argmax(pred[0])

# Выводим русское название класса согласно найденному индексу
print(class_names_rus[class_index])


In [ ]:
print(class_names_rus[y_test[0]])

## Визуализация модели и работы ее отдельных слоев

In [ ]:
from tensorflow.keras.utils import plot_model

# Визуализируем архитектуру модели
plot_model(model, show_shapes=True, show_layer_names=True, dpi=96, to_file='model.png')


In [ ]:
import matplotlib.pyplot as plt

def visualize_layer_activations(model, input_image, layer_names):
    activation_model = tf.keras.models.Model(inputs=model.inputs, outputs=[layer.output for layer in model.layers])
    activations = activation_model.predict(input_image)

    for layer_name, activation in zip(layer_names, activations):
        if len(activation.shape) == 4:  # Если активация имеет 4 размерности (обычный сверточный слой)
            n_features = activation.shape[-1]
            size = int(n_features ** 0.5)
            display_grid = np.zeros((size * activation.shape[1], size * activation.shape[2]))

            for i in range(size):
                for j in range(size):
                    feature_map = activation[0, :, :, i * size + j]
                    display_grid[i * activation.shape[1]:(i + 1) * activation.shape[1],
                                 j * activation.shape[2]:(j + 1) * activation.shape[2]] = feature_map

            scale = 20. / n_features
            plt.figure(figsize=(scale * size, scale * size))
            plt.title(layer_name)
            plt.grid(False)
            plt.imshow(display_grid, aspect='auto', cmap='viridis')

        elif len(activation.shape) == 2:  # Если активация имеет 2 размерности (слой Flatten)
            plt.figure()
            plt.title(layer_name)
            plt.grid(False)
            plt.imshow(activation, cmap='viridis', aspect='auto')

# Пример использования функции visualize_layer_activations
input_image = X_test[0:1]
layer_names = [layer.name for layer in model.layers]
visualize_layer_activations(model, input_image, layer_names)


In [ ]:
model.save("model.h5")